## Deploy BLIP2 Endpoint on SageMaker

In this notebook, we will deploy a BLIP2 endpoint with DJLServing container image.

### Setup

In [ ]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
import json
import base64

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

In [ ]:
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = "blip2"  # folder within bucket where code artifact will go
s3_model_prefix = "model_blip2"  # folder within bucket where code artifact will go
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{model_bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

## Prepare inference script and container image

In [ ]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=sess.boto_session.region_name, version="0.22.1"
)
inference_image_uri

In [ ]:
%%writefile blip2/serving.properties
engine = Python
option.tensor_parallel_degree = 1
# option.s3url = {{s3url}}

In [ ]:
model_names = {
    "caption_model_name":'blip2-2.7b', #@param ["blip-base", "blip-large", "git-large-coco"]
}
with open("blip2/model_name.json",'w') as file:
    json.dump(model_names, file)

In [ ]:
%%sh
tar czvf model.tar.gz blip2/
rm -rf blip2

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

## Deploy model

In [ ]:
from sagemaker.model import Model
from sagemaker.utils import name_from_base

model_name = name_from_base("blip2-27b")
model = Model(
    image_uri=inference_image_uri,
    model_data=s3_code_artifact,
    role=role,
    name=model_name,
)

In [ ]:
model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name=model_name
)

## Test Inference Endpoint

In [ ]:
from PIL import Image
import base64
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")
endpoint_name = model.endpoint_name

In [ ]:
def encode_image(img_file):
    with open(img_file, "rb") as image_file:
        img_str = base64.b64encode(image_file.read())
        base64_string = img_str.decode("latin1")
    return base64_string

def run_inference(endpoint_name, inputs):
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name, Body=json.dumps(inputs)
    )
    return response["Body"].read().decode('utf-8')

In [ ]:
test_image = "carcrash-ai.jpeg"
raw_image = Image.open(test_image).convert('RGB')
display(raw_image)

### Image captioning

In [ ]:
base64_string = encode_image(test_image)
inputs = {"image": base64_string}
run_inference(endpoint_name, inputs)

### Instructed zero-shot vision-to-language generation

In [ ]:
base64_string = encode_image('000000039769.jpg')
inputs = {"prompt": "Question: what are cats doing in this photo? Answer:", "image": base64_string}
run_inference(endpoint_name, inputs)

In [ ]:
context = [
    ("what are cats doing in this photo?", "sleeping"),
]
question = "where are two cats sleeping?"
template = "Question: {} Answer: {}."

prompt = " ".join([template.format(context[i][0], context[i][1]) for i in range(len(context))]) + " Question: " + question + " Answer:"
inputs = {"prompt": prompt, "image": base64_string}
run_inference(endpoint_name, inputs)

## Clean up

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)